### TELEMETRY TANQUES

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
df=pd.read_excel('Telemetry.xlsx')

In [3]:
df.columns=['Timestamp','Nivel (%)','Presion (bar)']

In [4]:
df['Date'] = [d.date() for d in df['Timestamp']]
df['Time'] = [d.time() for d in df['Timestamp']]

In [5]:
df = df.reindex(columns=['Timestamp','Date','Time','Nivel (%)','Presion (bar)'])
df=df.sort_values(by=['Date','Time'])
df=df.reset_index(drop=True)

In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
df['Year'] = df['Date'].dt.strftime('%Y')
df['Month'] = df['Date'].dt.strftime('%m')
df['Day'] = df['Date'].dt.strftime('%d')
df['Day_of_Week'] = df['Date'].dt.day_name()
df['Month_Name'] = df['Date'].dt.month_name()

In [8]:
def season_selector(date):
    season=['Winter','April','Summer','Autum']
    date=datetime.now()
    day=int(date.strftime("%d"))
    month=int(date.strftime("%m"))
    if (month >=1 and month <3) or (month ==12 and day >=21) or (month==3 and day < 21) :
        return season[0]
    if (month >3 and month < 6) or (month==6 and day<21) or (month==3 and day < 21):
        return season[1]
    if (month >6 and month < 9) or (month==6 and day>=21) or (month==9 and day<21):
        return season[2]
    if (month >9 and month < 12) or (month==9 and day >=21) or (month==12 and day<21):
        return season[3]

df['Season'] = df['Date'].apply(season_selector)

In [11]:
def inicio(nivel):
    
    num_valores=int(len(df['Nivel (%)']))
    
    if nivel == df['Nivel (%)'].iloc[0]:
        return "Inicio"
    if nivel == df['Nivel (%)'].iloc[(num_valores-1)]:
        return "Fin"
df['Estado']=df['Nivel (%)'].apply(inicio)

In [12]:
#Creamos lista con todos los niveles:
niveles=[nivel for nivel in df['Nivel (%)']]

In [13]:
#Creamos una función que nos indica si el valor de nivel corresponde a un estado de consumo o llenado de la cisterna:

def descarga_consumo():
    
    operacion=['inicio']
    n=1
    while n <(len(niveles)-1):
        if niveles[n] > niveles[n+1] and niveles[n]< niveles[n-1]:
            operacion.append("consumo")
            #print(f"{niveles[n]} > {niveles[n+1]} y {niveles[n]} < {niveles[n-1]}")
    
        elif niveles[n] < niveles[n+1] and niveles[n]> niveles[n-1]:
            operacion.append("descarga")
            #print(f"{niveles[n]} < {niveles[n+1]} y {niveles[n]} > {niveles[n-1]}")
        
        elif niveles[n] < niveles[n+1] and niveles[n]< niveles[n-1]:
            operacion.append("empieza descarga")
            #print(f"{niveles[n]} < {niveles[n+1]} y {niveles[n]} < {niveles[n-1]}")
        
        elif niveles[n] > niveles[n+1] and niveles[n]> niveles[n-1]:
            operacion.append("termina descarga")
            #print(f"{niveles[n]}> {niveles[n+1]} y {niveles[n]} > {niveles[n-1]}")
        
        elif niveles[n] == niveles[n+1]:
            #print(f"{niveles[n]} == {niveles[n+1]}")
            operacion.append("Sin consumo")
        
        elif niveles[n] == niveles[n-1]:
            #print(f"{niveles[n]} == {niveles[n-1]}")
            operacion.append("Sin consumo")
        n+=1
    return operacion

In [14]:
estado= descarga_consumo()

In [15]:
dff=df.copy()
mask=dff['Estado']
for n in range(len(estado)):
    mask.iloc[n]=estado[n]

/Users/javiertamarit/.local/share/virtualenvs/Ejercicios_Prog_Avanzada-OvvKQvod/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
#Filtra dataframe por descargas:
descargas_df = dff[dff['Estado'] == 'empieza descarga']
descargas_df =descargas_df.reset_index(drop=True)

In [17]:
numero_descargas = len(descargas_df)
print(numero_descargas)

30


In [18]:
# Calculamos las deltas de tiempo entre descargas y lo pasamos a una lista:

timestamps=[value for value in descargas_df['Timestamp']]

def delta_time_descargas(timestamps):
    
    delta_time=[0]
    
    for v in range(len(timestamps)-1):
        
        delta= timestamps[v+1]- timestamps[v]
        delta_time.append(delta)
        
    return delta_time

d=delta_time_descargas(timestamps)

In [19]:
def delta(timestamp):
    if timestamp == descargas_df['Timestamp'].iloc[0]:
        return 0
descargas_df['Delta Time']=descargas_df['Timestamp'].apply(delta)

In [ ]:
# Pasamos la lista de Delta Time al Dataframe:
n=1
mask=descargas_df['Delta Time']
while n < numero_descargas:
    mask.iloc[n]=d[n]
    n += 1   

In [ ]:
descargas_df